In [1]:
import pandas as pd
import numpy as np
import http.client
from dotenv import load_dotenv
import json
import ast  # To safely evaluate the string as a dictionary
import matplotlib.pyplot as plt
import seaborn as sns
import pycountry

import os
import time

In [2]:
# connect weather API

load_dotenv(dotenv_path=r"C:\Users\taoti\GitHub\Hotel_Recommendations\.env")


print(os.environ["weather_api_host"])
print(os.environ["weather_api_key"])

weatherbit-v1-mashape.p.rapidapi.com
a29fa7550emshd22ad2dcdc1d6fap1bfd73jsnbc24356674f8


In [3]:
# Construct City_country dataframe

# Create a mapping function for country abbreviations
def get_country_abbreviation(country_name):
    try:
         # use pycountry to get the country abbreviation
        country =  pycountry.countries.lookup(country_name)
        return country.alpha_2
    except LookupError:
         # If not found, return NaN
            # If not found, return NaN and print the country name
        #print(f"Country not found: {country_name}")
        return np.nan

# Get city_country dataframe
def city_country_dataframe():
    # load city, country dataframe
    city_country_df = pd.read_csv("worldcities.csv")

    # only select four colums:
    city_country_df = city_country_df[["city","lat","lng","country"]]

    # renmae column name
    city_country_df.rename(columns={"lng":"lon"},inplace=True)


    #apply(get_country_abbreviation) will pass each country name to the get_country_abbreviation function, which will return the 2-letter country code (if available).
    city_country_df["country_abbriev"] = city_country_df["country"].apply(get_country_abbreviation)
    # USA, UK
    city_country_df["country_abbriev"] = city_country_df["country_abbriev"].replace("US","USA") 
    city_country_df["country_abbriev"] = city_country_df["country_abbriev"].replace("GB","UK")      

    city_country_df.to_csv("city_country.csv")
    return city_country_df



# input city, country, then get location values
def input_city_country(city_country_df):
    # input city, country
    while True:
        # Get valid city
        city = input("Please input the city: ").replace(" ","")
        if not city_country_df["city"].str.contains(city,case=False,regex=False).any():
            print("City not found! Please input a valid city!")
        else:
            break
      
    
    while True:
        # Get valid country
        country = input("Please input the country (or abbreviation): ").replace(" ","")
        country_match = (city_country_df["country"].str.contains(country,case=False,regex=False)) | (city_country_df["country_abbriev"].str.upper() == country.upper())
        if not country_match.any():
            print("Country not found! Please input a valid country!")
        else:
            break
 
    
    # Filter for the specific city and country
    fil_location = (
        (city_country_df["city"].str.lower()==city.lower()) & 
        (
            (city_country_df["country"].str.lower()==country.lower()) | 
            (city_country_df["country_abbriev"]==country.upper())
         )
    )

    # Extract latitude and longitude as scalar values
    if fil_location.any():  # Check if the city-country combination exists
        location_data = city_country_df.loc[fil_location].iloc[0]

        lat = location_data["lat"]  # Extract value
        lon = location_data["lon"]  # Extract value
        city = location_data["city"]
        country = location_data["country"]

        location = (city,country,lat,lon)
    
        print(F"The destination is: ({city}, {country}) --> Latitude and longitude: ({lat},{lon})")
        return location        
    else:
        print(f"No data found for the location: {city}, {country}!")
        return  None


In [4]:

def get_url(location):
    city = location[0]
    country = location[1]
    lat = location[2]
    lon = location[3]
    # get url variable
    url = f"/forecast/daily?lat={lat}&lon={lon}&units=metric&lang=en"
    return url 


# define function to extract data from API given url, conn
def extract_weather_API(city,url,conn,headers):
    try: 
        # Request to the API, and get data as the response
        conn.request("GET", url, headers=headers)
        res = conn.getresponse()
        data = res.read()

        # Parse JSON and extract data
        weather_data = json.loads(data)
        print(f"Response for {city}: {weather_data}")
        
        if 'data' in weather_data:
                records = weather_data['data']
                # Add city information to each record
                for record in records:
                    record['city'] = city
                all_weather_data.extend(records)
        else:
                print(f"No 'data' key found for {city}. Skipping...")
                time.sleep(1)  # Wait 1 second between requests
    except Exception as e:
        print(f"Error fetching data for {city}: {e}")
        time.sleep(5)  # Wait 5 seconds before retrying
    
    return all_weather_data

In [5]:
# Main function

# Initialize connection
conn = http.client.HTTPSConnection(os.environ["weather_api_host"])
headers = {
    'x-rapidapi-key': os.environ["weather_api_key"],
    'x-rapidapi-host': os.environ["weather_api_host"]
}

# get city_country_df
city_country_df = city_country_dataframe()
# input city, country
location = input_city_country(city_country_df)
# get url 
weather_url = get_url(location)

# Create a list all_weather_data to store all records
all_weather_data = []
all_weather_data = extract_weather_API(location[0],weather_url,conn,headers)

# store data requested from API into dataframe for the convinience of analysis
# Create DataFrame from lsit all_weather_data
df = pd.DataFrame(all_weather_data)
# Display the DataFrame
print(df.head())
# Save the DataFrame to a CSV file
df.to_csv("weather_data_country.csv", index=False)

The destination is: (Madrid, Spain) --> Latitude and longitude: (40.4169,-3.7033)
Response for Madrid: {'city_name': 'Madrid', 'country_code': 'ES', 'data': [{'app_max_temp': 14.8, 'app_min_temp': 12.2, 'clouds': 42, 'clouds_hi': 52, 'clouds_low': 11, 'clouds_mid': 67, 'datetime': '2024-11-24', 'dewpt': 8.7, 'high_temp': 13.1, 'low_temp': 12.2, 'max_dhi': None, 'max_temp': 14.8, 'min_temp': 12.2, 'moon_phase': 0.27, 'moon_phase_lunation': 0.79, 'moonrise_ts': 1732408231, 'moonset_ts': 1732456701, 'ozone': None, 'pop': 10, 'precip': 0.17578125, 'pres': 943, 'rh': 73, 'slp': 1019, 'snow': 0, 'snow_depth': 0, 'sunrise_ts': 1732432238, 'sunset_ts': 1732467011, 'temp': 13.5, 'ts': 1732431660, 'uv': 0, 'valid_date': '2024-11-24', 'vis': None, 'weather': {'description': 'Broken clouds', 'code': 803, 'icon': 'c03d'}, 'wind_cdir': 'NE', 'wind_cdir_full': 'northeast', 'wind_dir': 53, 'wind_gust_spd': 5.3, 'wind_spd': 1.8}, {'app_max_temp': 14.8, 'app_min_temp': 9, 'clouds': 54, 'clouds_hi': 29, 